In [1]:
import sys
# 프로젝트 루트 직접 지정
sys.path.insert(0, r"c:\Users\tjrrj\vscode\doritest")

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd


In [3]:
"""
구글시트 추출(읽기) 함수 - 단일 진입점 extract_gsheet
파일명: modules/extract/extract_gsheet.py
"""

import re
from typing import Optional
from pathlib import Path
import pandas as pd


# ===== 기본 설정 =====
DEFAULT_CREDENTIALS = r"C:\Users\tjrrj\vscode\doritest\config\glowing-palace-465904-h6-7f82df929812.json"
DEFAULT_SHEET = "시트1"


def extract_gsheet(
    sheet_name: Optional[str] = None,
    *,
    url: Optional[str] = None,
    spreadsheet_id: Optional[str] = None,
    credentials_path: Optional[str] = None,
) -> pd.DataFrame:
    """
    구글시트에서 데이터 읽기 (단일 함수)

    Args:
        sheet_name: 시트명 (기본값: "시트1")
        url: 구글시트 공유 URL
        spreadsheet_id: 구글시트 ID (URL 대신 직접 전달 가능)
        credentials_path: 서비스 계정 JSON 경로

    Returns:
        pd.DataFrame

    Example:
        >>> df = extract_gsheet(
        ...     sheet_name="raw",
        ...     url="https://docs.google.com/spreadsheets/d/xxxx/edit"
        ... )
    """

    # ===== 라이브러리 체크 =====
    try:
        import gspread
        from google.oauth2 import service_account
    except ImportError as e:
        raise ImportError(
            "필수 라이브러리가 없습니다. "
            "pip install gspread google-auth"
        ) from e

    # ===== 기본값 =====
    sheet_name = sheet_name or DEFAULT_SHEET
    credentials_path = credentials_path or DEFAULT_CREDENTIALS

    # ===== 자격증명 파일 체크 =====
    if not Path(credentials_path).exists():
        raise FileNotFoundError(
            f"서비스 계정 JSON 파일이 존재하지 않습니다: {credentials_path}"
        )

    # ===== spreadsheet_id 결정 =====
    if not spreadsheet_id:
        if not url:
            raise ValueError(
                "url 또는 spreadsheet_id 중 하나는 반드시 전달해야 합니다."
            )

        # URL에서 /d/<spreadsheet_id> 추출
        m = re.search(r"/d/([a-zA-Z0-9_-]+)", url)
        if not m:
            raise ValueError("올바른 구글시트 URL 형식이 아닙니다.")
        spreadsheet_id = m.group(1)

    # ===== 인증 =====
    SCOPES = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    try:
        creds = service_account.Credentials.from_service_account_file(
            credentials_path, scopes=SCOPES
        )
        client = gspread.authorize(creds)
        spreadsheet = client.open_by_key(spreadsheet_id)
    except Exception as e:
        raise RuntimeError(f"구글시트 인증 또는 접근 실패: {e}") from e

    # ===== 시트 읽기 =====
    try:
        print(f"📖 구글시트 읽기: sheet='{sheet_name}'")
        worksheet = spreadsheet.worksheet(sheet_name)
        df = pd.DataFrame(worksheet.get_all_records())
        print(f"✅ 완료: {len(df)} rows")
        return df
    except Exception as e:
        raise RuntimeError(f"시트 읽기 실패: {e}") from e


In [4]:
extract_gsheet(url="https://docs.google.com/spreadsheets/d/17jloeN0cqpQrPAfcBaINjU4u02gMXgRfGfpvZ7uMj-E/edit?usp=sharing")

📖 구글시트 읽기: sheet='시트1'
✅ 완료: 12 rows


,order_id,주문번호,일자,시간,주문금액,업로드시간
0,69af3c645c335122,1CELLL,2025-11-30,2025-11-30 13:39:08,16600,2025-12-11 23:55:00
1,eb8f4cf6e038ab31,11L7JN,2025-11-30,2025-11-30 13:49:24,20000,2025-12-11 23:55:00
2,25501f4c34e41650,2DEKBH,2025-11-01,2025-11-01 13:41:28,24100,2025-12-13 05:32:03
3,6c8f45025bbee74e,0YEE8K,2025-11-01,2025-11-01 14:49:24,56100,2025-12-13 05:32:03
4,046b9cb1a9cc8dbe,29F98W,2025-11-01,2025-11-01 16:51:33,16100,2025-12-13 05:32:03
5,dd24c62c2bbc9d22,1TY694,2025-11-01,2025-11-01 17:39:05,21400,2025-12-13 05:32:03
6,45116497b2250554,28MJJK,2025-11-01,2025-11-01 18:25:45,16000,2025-12-13 05:32:03
7,25501f4c34e41650,2DEKBH,2025-11-01,2025-11-01 13:41:28,24100,2025-12-13 05:32:03
8,6c8f45025bbee74e,0YEE8K,2025-11-01,2025-11-01 14:49:24,56100,2025-12-13 05:32:03
9,046b9cb1a9cc8dbe,29F98W,2025-11-01,2025-11-01 16:51:33,16100,2025-12-13 05:32:03


In [12]:
"""
구글시트 추출(읽기) 함수 - 단일 진입점 extract_gsheet
파일명 기준 인터페이스 (시트명 미사용)
"""

import re
from typing import Optional
from pathlib import Path
import pandas as pd


# ===== 기본 설정 =====
DEFAULT_CREDENTIALS = r"C:\Users\tjrrj\vscode\doritest\config\glowing-palace-465904-h6-7f82df929812.json"


def extract_gsheet(
    *,
    file_name: Optional[str] = None,
    url: Optional[str] = None,
    spreadsheet_id: Optional[str] = None,
    credentials_path: Optional[str] = None,
) -> pd.DataFrame:
    """
    구글시트 파일에서 데이터 읽기 (파일명 + URL 기반)

    Args:
        file_name: 구글시트 파일명 (검증/로그용, 선택)
        url: 구글시트 공유 URL
        spreadsheet_id: 구글시트 ID (URL 대신 사용 가능)
        credentials_path: 서비스 계정 JSON 경로

    Returns:
        pd.DataFrame
    """

    # ===== 라이브러리 체크 =====
    try:
        import gspread
        from google.oauth2 import service_account
    except ImportError as e:
        raise ImportError(
            "pip install gspread google-auth 필요"
        ) from e

    # ===== credentials =====
    credentials_path = credentials_path or DEFAULT_CREDENTIALS
    if not Path(credentials_path).exists():
        raise FileNotFoundError(
            f"서비스 계정 JSON 파일이 존재하지 않습니다: {credentials_path}"
        )

    # ===== spreadsheet_id 결정 =====
    if not spreadsheet_id:
        if not url:
            raise ValueError(
                "url 또는 spreadsheet_id 중 하나는 반드시 필요합니다."
            )

        m = re.search(r"/d/([a-zA-Z0-9_-]+)", url)
        if not m:
            raise ValueError("올바른 구글시트 URL 형식이 아닙니다.")
        spreadsheet_id = m.group(1)

    # ===== 인증 =====
    SCOPES = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    try:
        creds = service_account.Credentials.from_service_account_file(
            credentials_path, scopes=SCOPES
        )
        client = gspread.authorize(creds)
        spreadsheet = client.open_by_key(spreadsheet_id)
    except Exception as e:
        raise RuntimeError(f"구글시트 인증/접근 실패: {e}") from e

    # ===== 파일명 검증 (선택) =====
    if file_name:
        actual_name = spreadsheet.title
        if actual_name != file_name:
            raise ValueError(
                f"파일명 불일치: 전달='{file_name}', 실제='{actual_name}'"
            )

    # ===== 첫 번째 시트 읽기 =====
    try:
        worksheet = spreadsheet.get_worksheet(0)
        df = pd.DataFrame(worksheet.get_all_records())
        return df
    except Exception as e:
        raise RuntimeError(f"시트 읽기 실패: {e}") from e


In [13]:
extract_gsheet(file_name="test_엑셀",
    url="https://docs.google.com/spreadsheets/d/17jloeN0cqpQrPAfcBaINjU4u02gMXgRfGfpvZ7uMj-E/edit?usp=sharing")

,order_id,주문번호,일자,시간,주문금액,업로드시간
0,69af3c645c335122,1CELLL,2025-11-30,2025-11-30 13:39:08,16600,2025-12-11 23:55:00
1,eb8f4cf6e038ab31,11L7JN,2025-11-30,2025-11-30 13:49:24,20000,2025-12-11 23:55:00
2,25501f4c34e41650,2DEKBH,2025-11-01,2025-11-01 13:41:28,24100,2025-12-13 05:32:03
3,6c8f45025bbee74e,0YEE8K,2025-11-01,2025-11-01 14:49:24,56100,2025-12-13 05:32:03
4,046b9cb1a9cc8dbe,29F98W,2025-11-01,2025-11-01 16:51:33,16100,2025-12-13 05:32:03
5,dd24c62c2bbc9d22,1TY694,2025-11-01,2025-11-01 17:39:05,21400,2025-12-13 05:32:03
6,45116497b2250554,28MJJK,2025-11-01,2025-11-01 18:25:45,16000,2025-12-13 05:32:03
7,25501f4c34e41650,2DEKBH,2025-11-01,2025-11-01 13:41:28,24100,2025-12-13 05:32:03
8,6c8f45025bbee74e,0YEE8K,2025-11-01,2025-11-01 14:49:24,56100,2025-12-13 05:32:03
9,046b9cb1a9cc8dbe,29F98W,2025-11-01,2025-11-01 16:51:33,16100,2025-12-13 05:32:03


# 코드

In [ ]:
"""
구글시트 추출(읽기) 함수 - 단일 진입점 extract_gsheet
파일명 기준 인터페이스 (시트명 미사용)
"""

import re
from typing import Optional
from pathlib import Path
import pandas as pd


# ===== 기본 설정 =====
DEFAULT_CREDENTIALS = '/opt/airflow/config/glowing-palace-465904-h6-7f82df929812.json'


def extract_gsheet(
    *,
    file_name: Optional[str] = None,
    url: Optional[str] = None,
    spreadsheet_id: Optional[str] = None,
    credentials_path: Optional[str] = None,
) -> pd.DataFrame:
    """
    구글시트 파일에서 데이터 읽기 (파일명 + URL 기반)

    Args:
        file_name: 구글시트 파일명 (검증/로그용, 선택)
        url: 구글시트 공유 URL
        spreadsheet_id: 구글시트 ID (URL 대신 사용 가능)
        credentials_path: 서비스 계정 JSON 경로

    Returns:
        pd.DataFrame
    """

    # ===== 라이브러리 체크 =====
    try:
        import gspread
        from google.oauth2 import service_account
    except ImportError as e:
        raise ImportError(
            "pip install gspread google-auth 필요"
        ) from e

    # ===== credentials =====
    credentials_path = credentials_path or DEFAULT_CREDENTIALS
    if not Path(credentials_path).exists():
        raise FileNotFoundError(
            f"서비스 계정 JSON 파일이 존재하지 않습니다: {credentials_path}"
        )

    # ===== spreadsheet_id 결정 =====
    if not spreadsheet_id:
        if not url:
            raise ValueError(
                "url 또는 spreadsheet_id 중 하나는 반드시 필요합니다."
            )

        m = re.search(r"/d/([a-zA-Z0-9_-]+)", url)
        if not m:
            raise ValueError("올바른 구글시트 URL 형식이 아닙니다.")
        spreadsheet_id = m.group(1)

    # ===== 인증 =====
    SCOPES = [
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive",
    ]

    try:
        creds = service_account.Credentials.from_service_account_file(
            credentials_path, scopes=SCOPES
        )
        client = gspread.authorize(creds)
        spreadsheet = client.open_by_key(spreadsheet_id)
    except Exception as e:
        raise RuntimeError(f"구글시트 인증/접근 실패: {e}") from e

    # ===== 파일명 검증 (선택) =====
    if file_name:
        actual_name = spreadsheet.title
        if actual_name != file_name:
            raise ValueError(
                f"파일명 불일치: 전달='{file_name}', 실제='{actual_name}'"
            )

    # ===== 첫 번째 시트 읽기 =====
    try:
        worksheet = spreadsheet.get_worksheet(0)
        df = pd.DataFrame(worksheet.get_all_records())
        return df
    except Exception as e:
        raise RuntimeError(f"시트 읽기 실패: {e}") from e
